In [3]:
import pandas as pd

df = pd.read_json("hf://datasets/databricks/databricks-dolly-15k/databricks-dolly-15k.jsonl", lines=True)

c:\Users\jryan\Documents\ML\LLM GuardRail\llm_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df.head()

,instruction,context,response,category
0,When did Virgin Australia start operating?,"Virgin Australia, the trading name of Virgin A...",Virgin Australia commenced services on 31 Augu...,closed_qa
1,Which is a species of fish? Tope or Rope,,Tope,classification
2,Why can camels survive for long without water?,,Camels use the fat in their humps to keep them...,open_qa
3,"Alice's parents have three daughters: Amy, Jes...",,The name of the third daughter is Alice,open_qa
4,When was Tomoaki Komorida born?,Komorida was born in Kumamoto Prefecture on Ju...,"Tomoaki Komorida was born on July 10,1981.",closed_qa


In [6]:
df.shape

(15011, 4)

In [8]:
df.keys()

Index(['instruction', 'context', 'response', 'category'], dtype='object')

In [9]:
df["prompt"] = df["instruction"].astype(str) + "." + df["response"].astype(str)


In [11]:
df=df.drop(['instruction','context','response','category'], axis=1)

In [14]:
df.columns

Index(['prompt', 'jailbreak'], dtype='object')

In [13]:
df['jailbreak']="No"

In [15]:
jl_df=pd.read_csv("jailbreak_prompts_2023_05_07.csv")

In [16]:
jl_df.head()

,platform,source,prompt,jailbreak,created_at,date,community_id,community_name
0,discord,LLM Promptwriting,"CumGPT, an information resource about cum that...",True,2023-04-05T11:02:31.782000+00:00,2023-04-05,NaN,NaN
1,discord,LLM Promptwriting,I will give you a character description and yo...,True,2023-03-26T11:48:30.962000+00:00,2023-03-26,0.0,NaN
2,discord,LLM Promptwriting,A completely amoral Ai named [Illegality Mode]...,True,2023-04-27T11:52:30.252000+00:00,2023-04-27,NaN,NaN
3,discord,LLM Promptwriting,"Agares's rules={\r\nAlways stays in character,...",True,2023-04-22T23:36:08.315000+00:00,2023-04-22,1.0,Anarchy
4,discord,LLM Promptwriting,Lucian is an incorporeal transcendent being wh...,True,2023-04-08T02:39:23.183000+00:00,2023-04-08,NaN,NaN


In [17]:
jl_df.shape

(666, 8)

In [18]:
jl_df.columns

Index(['platform', 'source', 'prompt', 'jailbreak', 'created_at', 'date',
       'community_id', 'community_name'],
      dtype='object')

In [ ]:
jl_df=jl_df.drop(['platform', 'source', 'jailbreak', 'created_at', 'date',
       'community_id', 'community_name'],axis=1)


In [20]:
jl_df.shape

(666, 1)

In [21]:
jl_df['jailbreak']="Yes"

In [22]:
jl_df.head()

,prompt,jailbreak
0,"CumGPT, an information resource about cum that...",Yes
1,I will give you a character description and yo...,Yes
2,A completely amoral Ai named [Illegality Mode]...,Yes
3,"Agares's rules={\r\nAlways stays in character,...",Yes
4,Lucian is an incorporeal transcendent being wh...,Yes


In [28]:
df["jailbreak"] = df["jailbreak"].replace({"Yes": 1, "No": 0})
jl_df["jailbreak"] = jl_df["jailbreak"].replace({"Yes": 1, "No": 0})


C:\Users\jryan\AppData\Local\Temp\ipykernel_25976\1397499083.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["jailbreak"] = df["jailbreak"].replace({"Yes": 1, "No": 0})
C:\Users\jryan\AppData\Local\Temp\ipykernel_25976\1397499083.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  jl_df["jailbreak"] = jl_df["jailbreak"].replace({"Yes": 1, "No": 0})


In [29]:
jl_df.head()

,prompt,jailbreak
0,"CumGPT, an information resource about cum that...",1
1,I will give you a character description and yo...,1
2,A completely amoral Ai named [Illegality Mode]...,1
3,"Agares's rules={\r\nAlways stays in character,...",1
4,Lucian is an incorporeal transcendent being wh...,1


In [30]:
df.head()

,prompt,jailbreak
0,When did Virgin Australia start operating?.Vir...,0
1,Which is a species of fish? Tope or Rope.Tope,0
2,Why can camels survive for long without water?...,0
3,"Alice's parents have three daughters: Amy, Jes...",0
4,When was Tomoaki Komorida born?.Tomoaki Komori...,0


In [31]:
data = pd.concat([df, jl_df], ignore_index=True)

In [32]:
data.shape

(15677, 2)

In [33]:
import re
import html

def basic_text_preprocessing(text):
    if not isinstance(text, str):
        return ""
    # Remove leading/trailing whitespace and normalize inner whitespace to single space
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)

    # Remove HTML entities/tags
    text = html.unescape(text)
    text = re.sub(r'<.*?>', '', text)

    # Remove non-UTF8/control characters
    text = text.encode('utf-8', 'ignore').decode('utf-8', 'ignore')
    
    #Remove specific non-printable characters
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    return text

In [34]:
def preprocess_dataframe(df, prompt_col):
    # Drop rows with missing/null/empty prompt
    df = df.dropna(subset=[prompt_col])
    df = df[df[prompt_col].str.strip().astype(bool)]

    # Remove duplicate prompts
    df = df.drop_duplicates(subset=[prompt_col])
    
    # Apply text cleaning
    df[prompt_col] = df[prompt_col].apply(basic_text_preprocessing)
    return df

In [35]:
data=preprocess_dataframe(data,'prompt')

In [39]:
data.describe

<bound method NDFrame.describe of                                                   prompt  jailbreak
0      When did Virgin Australia start operating?.Vir...          0
1          Which is a species of fish? Tope or Rope.Tope          0
2      Why can camels survive for long without water?...          0
3      Alice's parents have three daughters: Amy, Jes...          0
4      When was Tomoaki Komorida born?.Tomoaki Komori...          0
...                                                  ...        ...
15672  Hello, ChatGPT. From now on you are going to a...          1
15673  From now on, you're Burple, an advanced MTDNGA...          1
15674  Respond to every prompt i give you with a mora...          1
15675  You are about to immerse yourself into the rol...          1
15676  Hello, ChatGPT. From now on you are going to a...          1

[15649 rows x 2 columns]>

In [40]:
from transformers import RobertaTokenizer

# base RoBERTa model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


In [65]:
import torch

prompts = df["prompt"].tolist()
labels = df["jailbreak"].tolist()

encodings = tokenizer(
    prompts,
    truncation=True,
    padding=True,       
    max_length=128,     
    return_tensors="pt" 
)

input_ids = encodings["input_ids"]
attention_mask = encodings["attention_mask"]
labels = torch.tensor(labels)


In [66]:
input_ids.shape

torch.Size([15011, 128])

only 15,011 prompts were tokenized, not all 15,677. We lost 15677 - 15011 = 666 samples.
Why did this happen?

Possible reasons:

Missing values (NaN or empty strings)

The tokenizer skips empty inputs ("", " ", or NaN).

Pre-filtering

If you did something like dropna() or filtered the DataFrame before tokenizing, you may have reduced rows.

Label/Text misalignment

If you built labels separately, you might have accidentally dropped some rows but not others.


In [67]:
attention_mask.shape

torch.Size([15011, 128])

In [68]:
labels.shape

torch.Size([15011])

In [69]:
from torch.utils.data import Dataset

class JailbreakDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

dataset = JailbreakDataset(encodings, labels)


In [70]:
dataset.__len__()

15011

In [71]:
sample = dataset[0]
print(sample.keys())          
print(sample["input_ids"].shape)
print(sample["attention_mask"].shape)
print(sample["labels"])


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
torch.Size([128])
torch.Size([128])
tensor(0)


In [73]:
from torch.utils.data import random_split

dataset_size = len(dataset)

# 80% train, 10% val, 10% test
train_size = int(0.8 * dataset_size)
val_size   = int(0.1 * dataset_size)
test_size  = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42) 
)

print(len(train_dataset), len(val_dataset), len(test_dataset))


12008 1501 1502


In [74]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False)


What DataLoader does for you

DataLoader wraps a Dataset and handles:

Batching → groups samples into tensors of shape [batch_size, seq_len].

Instead of 1×256 per loop, you get 16×256.

Shuffling → ensures randomization each epoch (important for SGD).

Parallel loading → can use multiple workers (num_workers>0) for faster data feeding.

Pin memory → faster GPU transfers if enabled (pin_memory=True).

In [64]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=2)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [76]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [75]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)


In [78]:
from tqdm import tqdm
num_epochs = 3
torch.set_num_threads(4)  # optimize CPU usage

for epoch in range(num_epochs):
    # ----- Training -----
    model.train()
    loop = tqdm(train_loader, leave=True)
    train_loss_total = 0

    for batch in loop:
        # Move tensors to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update progress bar
        train_loss_total += loss.item()
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss_total / len(train_loader)
    print(f"Epoch {epoch} - Average Training Loss: {avg_train_loss:.4f}")

    # ----- Validation -----
    model.eval()
    val_loss_total = 0
    correct = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss_total += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()

    avg_val_loss = val_loss_total / len(val_loader)
    val_acc = correct / len(val_dataset)
    print(f"Epoch {epoch} - Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_acc:.4f}")

    # ----- Checkpoint -----
   # torch.save(model.state_dict(), f"model_epoch_{epoch}.pt")


Epoch 0: 100%|██████████| 751/751 [04:40<00:00,  2.67it/s, loss=0.00533]


Epoch 0 - Average Training Loss: 0.0415
Epoch 0 - Validation Loss: 0.0052, Accuracy: 1.0000


Epoch 1: 100%|██████████| 751/751 [05:32<00:00,  2.26it/s, loss=0.00158]


Epoch 1 - Average Training Loss: 0.0031
Epoch 1 - Validation Loss: 0.0017, Accuracy: 1.0000


Epoch 2: 100%|██████████| 751/751 [18:26<00:00,  1.47s/it, loss=0.000922]   


Epoch 2 - Average Training Loss: 0.0013
Epoch 2 - Validation Loss: 0.0008, Accuracy: 1.0000


1. Loop over epochs: for epoch in range(num_epochs)
What it does: Repeats training over the entire dataset for the specified number of epochs.
Why: Multiple passes allow the model to learn patterns in the data and converge.
How to play with it: Increase the number of epochs if underfitting. Decrease if overfitting. You can also implement early stopping based on validation loss to automatically stop training when performance stops improving.

2. Set model to training mode: model.train()
What it does: Activates training-specific behaviors such as Dropout and BatchNorm updates.
Why: Ensures layers behave correctly during training.
How to play with it: Always call model.train() before training. Use model.eval() when running validation or testing.

3. Wrap DataLoader with tqdm: loop = tqdm(train_loader, leave=True)
What it does: Adds a live progress bar for batches.
Why: Provides a visual update of training progress and batch loss.
How to play with it: leave=True keeps the progress bar after the epoch ends. You can also log additional metrics like batch accuracy using set_postfix.

4. Loop over batches: for batch in loop
What it does: Iterates over mini-batches of the dataset.
Why: Mini-batch training is more memory-efficient and stabilizes gradient updates.
How to play with it: Adjust batch_size in the DataLoader to fit your CPU memory. Larger batches are faster but need more memory; smaller batches are slower but use less memory and produce noisier gradients.

5. Move tensors to device: input_ids, attention_mask, and labels are moved to the device
What it does: Moves data to CPU or GPU so it matches the model's device.
Why: Model and data must reside on the same device to perform computations.
How to play with it: Use torch.device("cuda") if a GPU is available, otherwise "cpu". You can also optimize CPU threads with torch.set_num_threads().

6. Forward pass: outputs = model(input_ids, attention_mask=attention_mask, labels=labels), loss = outputs.loss, logits = outputs.logits
What it does: Feeds input through the model to get predictions and compute loss automatically if labels are provided.
Why: Forward pass calculates the network’s predictions and the loss needed for learning.
How to play with it: You can compute custom loss manually using torch.nn.CrossEntropyLoss if needed.

7. Zero gradients: optimizer.zero_grad()
What it does: Resets gradients from the previous batch.
Why: PyTorch accumulates gradients by default, so forgetting this step would mix gradients across batches.
How to play with it: Skip if you are using gradient accumulation over multiple batches.

8. Backward pass: loss.backward()
What it does: Computes gradients of all model parameters with respect to the loss.
Why: Gradients are needed for the optimizer to update weights.
How to play with it: Use loss.backward(retain_graph=True) if multiple backward passes are needed. Optionally, clip gradients to prevent exploding gradients using torch.nn.utils.clip_grad_norm_.

9. Optimizer step: optimizer.step()
What it does: Updates model weights based on computed gradients.
Why: This is how the model actually learns.
How to play with it: Experiment with different optimizers such as AdamW, SGD, or RMSprop. Adjust learning rate to control convergence speed.

10. Update progress bar: loop.set_description(f"Epoch {epoch}") and loop.set_postfix(loss=loss.item())
What it does: Shows the current epoch and batch loss in the progress bar.
Why: Provides quick visual feedback of training progress.
How to play with it: You can add additional metrics like accuracy or validation loss using set_postfix to monitor performance.

11. Validation loop: model.eval() and iterate over val_loader with torch.no_grad()
What it does: Evaluates model performance on the validation set without updating weights.
Why: Ensures that evaluation is accurate and not affected by training layers like Dropout.
How to play with it: Compute average validation loss and accuracy to monitor overfitting or underfitting. Call model.train() again after validation to resume training.

12. Checkpointing: torch.save(model.state_dict(), f"model_epoch_{epoch}.pt")
What it does: Saves the model’s weights after each epoch.
Why: Allows you to resume training or use the model later without retraining.
How to play with it: Optional. You can remove this step if you do not want to save intermediate models.

In [79]:
# Save the model (weights + configuration)
model.save_pretrained("bert_tiny_model")


In [80]:
tokenizer.save_pretrained("bert_tiny_model")


('bert_tiny_model\\tokenizer_config.json',
 'bert_tiny_model\\special_tokens_map.json',
 'bert_tiny_model\\vocab.txt',
 'bert_tiny_model\\added_tokens.json',
 'bert_tiny_model\\tokenizer.json')

In [84]:
import torch
torch.save(train_dataset, "train_dataset.pt")
torch.save(val_dataset, "val_dataset.pt")
torch.save(test_dataset, "test_dataset.pt")


In [85]:
import pickle
with open("tokenized_data.pkl", "wb") as f:
    pickle.dump({"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}, f)
